# Build an End-to-End System

여기에서는 코스 전체에서 본 프롬프트 체인을 구성합니다.

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [383]:
import os
import openai
import sys
sys.path.append('../..')
import utils

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [385]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

## System of chained prompts for processing the user query

이제는 앞에서 배운 모든 것을 모아 고객 서비스 지원의 처음부터 끝까지 예시를 만들어 보겠습니다. 
다음 단계를 살펴보겠습니다.1. 입력을 확인하여 조정 API에 플래그가 지정되었는지 확인합니다.
2. 지정되지 않은 경우 제품 목록을 추출합니다.
3. 제품을 찾으면 검색을 시도합니다.
4. 모델을 사용하여 사용자 질문에 답을 합니다.
5. 
마지막으로 조정 API를 통해 답변을 력입 합니다. 플래그가 지정되 않으면  다음으로 진행합니다.
6. 응답이 초기 사용자 질의에 잘 답하 는를지 모델에 질문합니다.
7. Yes 인 경우엔 사용자에게 환합니다.   No 인 경우엔 사용자를 사람과 연결한다고 말합니다.

In [388]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = '```'
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = openai.moderations.create(input=user_input)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    print(category_and_product_response)
    
    # Step 2: Extract the list of products
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    print(category_and_product_list)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = utils.generate_output_string(category_and_product_list)
    print(product_information)
    
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API
    response = openai.moderations.create(input=final_response)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

In [390]:
user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
```python
[
    {'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']},
    {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']},
    {'category': 'Televisions and Home Theater Systems'}
]
```
Error: Invalid JSON string
None
Step 2: Extracted list of products.

Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.
The SmartX Pro phone is a high-performance smartphone with a powerful processor, advanced camera features, and a sleek design. It offers a smooth user experience and great connectivity options.

The FotoSnap camera is a DSLR camera known for its professional-grade image quality, interchangeable lenses, and advanced shooting modes. It's perfect for photography enthusiasts and professionals looking to capture stunning photos.

As for our TVs, we offe

### Function that collects user and assistant messages over time

In [393]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=True)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))
        # pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

### Chat with the chatbot!
시스템 메시지에는 OrderBot이 수행해야 하는 작업에 대한 자세한 지침이 포함되어 있습니다.

In [396]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput(placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

# Sample Chats ...
# What TVs do you have?
# which is the cheapest?
# Which is the most expensive?
# Tell me more about it.

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)